## Data Cleaning and Preprocessing Notebook

This notebook aims to cleana dataset that combines Environmental, Social, and Governance (ESG) scores with various financial metrics of companies. It also preprocess the data so that we have everything we need for the analysis part at the end of this notebook.

### Load packages and define the fucntions for data cleaning and preprocessing

In [1]:
import pandas as pd
import numpy as np

In [2]:
def classify_sic(sic_code):
    """
    Classify a given Standard Industrial Classification (SIC) code into a broad industry category.

    Parameters:
    - sic_code (int): The SIC code to classify. This should be a four-digit integer representing 
      the industry of a company or business entity.

    Returns:
    - str: A string representing the broad industry category. Possible return values include 
      'Agriculture, Forestry, and Fishing', 'Mining', 'Construction', 'Manufacturing', 
      'Transportation and Public Utilities', 'Wholesale Trade', 'Retail Trade', 
      'Finance, Insurance, Real Estate', 'Services', 'Public Administration', and 
      'Nonclassifiable' for SIC codes that do not fit into the other categories.
    """
    
    if 100 <= sic_code <= 999:
        return 'Agriculture, Forestry, and Fishing'
    elif 1000 <= sic_code <= 1499:
        return 'Mining'
    elif 1500 <= sic_code <= 1799:
        return 'Construction'
    elif 2000 <= sic_code <= 3999:
        return 'Manufacturing'
    elif 4000 <= sic_code <= 4999:
        return 'Transportation and Public Utilities'
    elif 5000 <= sic_code <= 5199:
        return 'Wholesale Trade'
    elif 5200 <= sic_code <= 5999:
        return 'Retail Trade'
    elif 6000 <= sic_code <= 6799:
        return 'Finance, Insurance, Real Estate'
    elif 7000 <= sic_code <= 8999:
        return 'Services'
    elif 9100 <= sic_code <= 9799:
        return 'Public Administration'
    else:
        return 'Nonclassifiable'

## Data Cleaning

In [3]:
data_with_esg_scores = pd.read_csv('data_with_esg_scores.csv',dtype={'Filename': str})
data_with_esg_scores.replace(["n.a.", "n.s."], pd.NA, inplace=True)
# Rows with missing values in critical financial metrics and ESG scores are removed
# to ensure the integrity of the analysis.
result_cleaned = data_with_esg_scores.dropna(subset=['ROE using P/L before tax\nLast avail. yr',
                                                     "Total assets\nth USD Last avail. yr",
                                                     "Solvency ratio (Asset based)\nLast avail. yr"])

## Data Preprocessing

In [4]:
# rename for convinence
result_cleaned = result_cleaned.rename(columns={'ROE using P/L before tax\nLast avail. yr': 'roe'})
result_cleaned = result_cleaned.rename(columns={"Total assets\nth USD Last avail. yr": "asset"})
result_cleaned = result_cleaned.rename(columns={"Solvency ratio (Asset based)\nLast avail. yr": "solvency"})
result_cleaned = result_cleaned.rename(columns={"US SIC, primary code(s)": "sic"})

In [5]:
result_cleaned = result_cleaned.rename(columns={"Total ESG Score": "Total ESG Sentiment"})
result_cleaned = result_cleaned.rename(columns={"Environmental Score": "Total Environmental Sentiment"})
result_cleaned = result_cleaned.rename(columns={"Social Score": "Total Social Sentiment"})
result_cleaned = result_cleaned.rename(columns={"Governance Score": "Total Governance Sentiment"})

In [6]:
df = result_cleaned[["roe", "asset", "solvency",'Total ESG Sentiment', "Total Environmental Sentiment", "Total Social Sentiment", "Total Governance Sentiment"]]

# Selected columns are converted to float type
df = df.astype(float)

df["Content"] = result_cleaned["Content"]
df["sic"] = result_cleaned["sic"]

In [7]:
# deal with outliers
df = df[(df['roe'] >= -100) & (df['roe'] <= 100)]

In [8]:
# construct variables
df['asset_log'] = np.log(df['asset'])
df["roe_asset_log"] = df["roe"] * df["asset_log"]
df["roe_solvency"] = df["roe"] * df["solvency"]
df["asset_log_solvency"] = df["asset_log"] * df["solvency"]

In [9]:
df.describe()

,roe,asset,solvency,Total ESG Sentiment,Total Environmental Sentiment,Total Social Sentiment,Total Governance Sentiment,sic,asset_log,roe_asset_log,roe_solvency,asset_log_solvency
count,984.000000,9.840000e+02,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000,984.000000
mean,9.007768,1.357856e+07,44.723943,0.143324,0.043546,0.043891,0.055886,4890.928862,14.931516,154.482975,258.029206,652.903784
std,29.537900,3.806494e+07,20.167788,0.052486,0.041821,0.023513,0.035427,2104.716903,1.761793,429.850339,1427.491015,272.517369
min,-96.581000,3.096520e+03,3.945000,0.000000,0.000000,0.000000,0.000000,174.000000,8.038034,-1401.308951,-8610.679055,65.474546
25%,-3.093500,9.931552e+05,29.958750,0.109048,0.010010,0.026291,0.035248,3491.000000,13.808631,-42.010371,-117.940164,457.068242
50%,10.586000,3.013561e+06,43.047500,0.142326,0.030798,0.041623,0.048708,4911.000000,14.918629,163.460060,389.347814,637.130502
75%,24.052500,9.293919e+06,57.451500,0.174789,0.064500,0.058521,0.066448,6798.000000,16.044871,367.081294,908.568794,847.303053
max,99.210000,5.278540e+08,99.922000,0.392510,0.241185,0.160381,0.320735,9711.000000,20.084330,1646.572801,4788.655040,1487.791602


There are altogether 984 valid samples. The above table displays everything needed for data analysis.

In [10]:
df['industry'] = df['sic'].apply(classify_sic)

In [12]:
# export data for further analysis
df.to_csv('data_cleaned.csv', index=False)